In [24]:
import pandas as pd
import os

Define Paths

In [44]:
TRANSFORMED_DATA_DIR = "../../data/silver"
MART_DATA_DIM_DIR = "../../data/gold/dimension"
MART_DATA_FACT_DIR = "../../data/gold/fact"
os.makedirs(MART_DATA_DIM_DIR, exist_ok=True)
os.makedirs(MART_DATA_FACT_DIR, exist_ok=True)

Load Transformed Data

In [28]:

tables = [
    "User", "Designation", "Course", "EmployeeProgress", 
    "Skill", "UserSkill", "DesignationSkill", "CourseSkill"
]


transformed_dfs = {}

for table in tables:
    file_path = os.path.join(TRANSFORMED_DATA_DIR, f'prep_{table}.csv')
    transformed_dfs[table] = pd.read_csv(file_path)


Create User Dimension Table

In [31]:
user_dim = transformed_dfs["User"][['id', 'username', 'createdAt']]
user_dim.columns = ['user_id', 'user_name', 'joined_at']  # Rename columns
print("User Dimension Preview:")
print(user_dim.head())


User Dimension Preview:
   user_id       user_name            joined_at
0        2    Allison Chan  2023-06-14 19:04:33
1        3    Scott Flores  2023-03-25 18:14:50
2        4  Frank Trujillo  2022-10-14 09:55:19
3        5   Dana Gonzalez  2022-06-27 08:07:37
4        6      Scott Holt  2023-02-03 17:27:47


Create Designation Dimension Table

In [32]:
designation_dim = transformed_dfs["Designation"][['id', 'name']]
designation_dim.columns = ['designation_id', 'designation_name']  # Rename columns
print("\nDesignation Dimension Preview:")
print(designation_dim.head())



Designation Dimension Preview:
   designation_id    designation_name
0               1  Software Developer
1               2       Data Engineer
2               3                  HR
3               4     Sales Executive
4               5       IT Specialist


Create Course Dimension Table

In [34]:
course_dim = transformed_dfs["Course"][['course_id', 'title', 'difficulty', 'language', 'totalTime', 'totalModules', 'created_at']]
print("\nCourse Dimension Preview:")
print(course_dim.head())



Course Dimension Preview:
   course_id                                      title    difficulty  \
0          1  Total solution-oriented Graphic Interface  Intermediate   
1          2          Enhanced mobile Graphic Interface  Intermediate   
2          3              Fundamental radical workforce      Beginner   
3          4                 Expanded exuding help-desk      Advanced   
4          5               Object-based global strategy      Advanced   

   language  totalTime  totalModules           created_at  
0  Japanese        379             6  2023-12-29 15:22:34  
1   Spanish        421            27  2022-10-28 02:22:40  
2   English        354            13  2023-02-25 19:00:50  
3     Hindi        274            30  2023-05-25 23:51:59  
4    German        193            29  2023-09-04 00:25:38  


Create Skill Dimension Table

In [36]:
skill_dim = transformed_dfs["Skill"][['id', 'name']]
skill_dim.columns = ['skill_id', 'skill_name'] 
print("\nSkill Dimension Preview:")
print(skill_dim.head())



Skill Dimension Preview:
   skill_id            skill_name
0         1            Leadership
1         2  CXO level engagement
2         3               Glamour
3         4                 IKEv2
4         5           Awesomeness


Create Employee Progress Fact Table

In [38]:
employee_progress = transformed_dfs["EmployeeProgress"].merge(transformed_dfs["User"], left_on='userId', right_on='id', how='left')


employee_progress = employee_progress.sort_values(by=['userId', 'courseId', 'last_updated'])

# Calculate the previous update date using shift
employee_progress['previous_update'] = employee_progress.groupby(['userId', 'courseId'])['last_updated'].shift()

employee_progress_fact = employee_progress[[
    'userId', 'courseId', 'designationId', 'progressStatus', 
    'last_updated', 'modulesCompleted', 'has_certificate', 
    'previous_update'
]]

print("\nEmployee Progress Fact Table Preview:")
print(employee_progress_fact.head())



Employee Progress Fact Table Preview:
   userId  courseId  designationId progressStatus             last_updated  \
2       2       119              5    in_progress  2024-07-18 11:08:39.000   
3       2       119              5      completed  2024-08-17 06:29:20.000   
0       2       331              5    in_progress  2024-01-18 01:59:25.000   
1       2       331              5      completed  2024-07-30 06:51:29.000   
6       3       341              7    in_progress  2023-12-05 03:41:02.000   

   modulesCompleted  has_certificate          previous_update  
2                 6                0                      NaN  
3                 8                1  2024-07-18 11:08:39.000  
0                 6                0                      NaN  
1                10                1  2024-01-18 01:59:25.000  
6                19                0                      NaN  


Create Course Enrollment Fact Table

In [39]:
course_enroll_fact = transformed_dfs["EmployeeProgress"]

course_enroll_fact['completed_and_has_certificate'] = (course_enroll_fact['progressStatus'] == 'completed') & (course_enroll_fact['has_certificate'] == True)

course_enroll_fact = course_enroll_fact.groupby('courseId').agg(
    count_of_enrolled=('userId', 'count'), 
    count_of_unique_users=('userId', 'nunique'), 
    count_of_completed_and_has_certificate=('completed_and_has_certificate', 'sum') 
).reset_index()

print("\nCourse Enrollment Fact Table Preview:")
print(course_enroll_fact.head())



Course Enrollment Fact Table Preview:
   courseId  count_of_enrolled  count_of_unique_users  \
0         1                  9                      3   
1         2                  5                      1   
2         3                 12                      5   
3         4                  2                      1   
4         5                  3                      1   

   count_of_completed_and_has_certificate  
0                                       2  
1                                       1  
2                                       4  
3                                       1  
4                                       1  


In [40]:
user_designation = transformed_dfs["User"].merge(transformed_dfs["Designation"], left_on='designationId', right_on='id', suffixes=('', '_designation'))
user_designation_skills = user_designation.merge(transformed_dfs["DesignationSkill"], left_on='designationId', right_on='designationId')
user_designation_skills = user_designation_skills.merge(transformed_dfs["Skill"], left_on='skillId', right_on='id', suffixes=('', '_skill'))
fact_table = user_designation_skills.merge(transformed_dfs["UserSkill"], left_on=['id', 'skillId'], right_on=['userId', 'skillId'], how='left', suffixes=('', '_user_skill'))

# Determine fulfillment status
fact_table['fulfillmentStatus'] = fact_table['userId'].apply(lambda x: 'Fulfilled' if pd.notnull(x) else 'Not Fulfilled')

skill_fulfillment_fact = fact_table[['id', 'username', 'designationId', 'name', 'skillId', 'name_skill', 'fulfillmentStatus']]

print(skill_fulfillment_fact)

       id      username  designationId               name  skillId  \
0       2  Allison Chan              5      IT Specialist       20   
1       2  Allison Chan              5      IT Specialist      150   
2       2  Allison Chan              5      IT Specialist       78   
3       2  Allison Chan              5      IT Specialist       16   
4       2  Allison Chan              5      IT Specialist      146   
...   ...           ...            ...                ...      ...   
2824  500  Sarah Greene              8  Financial Analyst      187   
2825  500  Sarah Greene              8  Financial Analyst       36   
2826  500  Sarah Greene              8  Financial Analyst      189   
2827  500  Sarah Greene              8  Financial Analyst      183   
2828  500  Sarah Greene              8  Financial Analyst      112   

                     name_skill fulfillmentStatus  
0          Network Architecture     Not Fulfilled  
1                           SQF     Not Fulfilled  
2  

In [45]:
user_dim.to_csv(os.path.join(MART_DATA_DIM_DIR, 'user_dimension.csv'), index=False)

designation_dim.to_csv(os.path.join(MART_DATA_DIM_DIR, 'designation_dimension.csv'), index=False)

course_dim.to_csv(os.path.join(MART_DATA_DIM_DIR, 'course_dimension.csv'), index=False)

skill_dim.to_csv(os.path.join(MART_DATA_DIM_DIR, 'skill_dimension.csv'), index=False)

employee_progress_fact.to_csv(os.path.join(MART_DATA_FACT_DIR, 'employee_progress_fact.csv'), index=False)

course_enroll_fact.to_csv(os.path.join(MART_DATA_FACT_DIR, 'course_enrollment_fact.csv'), index=False)

skill_fulfillment_fact.to_csv(os.path.join(MART_DATA_FACT_DIR, 'skill_fulfillment_fact.csv'), index=False)


Dimension Tables

1) User Dim: ['id', 'username', 'createdAt']

2) Designation Dim: ['id', 'name']

3) Course Dim: ['course_id', 'title', 'difficulty', 'language', 'totalTime', 'totalModules', 'created_at']

4) Skill Dim: ['id', 'name']


Fact Tables

1) EmployeeProgress Fact: ['userId', 'courseId', 'designationId', 'progressStatus', 'last_updated', 'modulesCompleted', 'has_certificate', 'previous_update']

2) CourseEnroll Fact: ['courseId', 'count_of_enrolled', 'count_of_completed_and_has_certificate'] 

3) Skill Fact: ['id', 'username', 'designationId', 'name', 'skillId', 'name_skill', 'fulfillmentStatus']